In [ ]:
inputdir = '/facility/imganfacusers/Ashesh/NatureMethodsSegmentation/2404_D21-M3-S0-L8_6/'
segmentationdir = '/facility/imganfacusers/Ashesh/NatureMethodsSegmentationOutputs/Analysis_2404_D21-m3-S0-L8_6/'
OUTPUT_DIR = '/group/jug/ashesh/naturemethods/segmentation/three_analysts/2404_D21-M3-S0-L8_6/'

In [ ]:
from disentangle.core.tiff_reader import load_tiff
import os
ch_idx = 0 # which channel are we interested in doing segmentation.
crop_size = 1600 # because the prediction is for top left 1600x1600, we need to crop the GT to match the size.
gt = load_tiff(os.path.join(inputdir, 'GT.tif'))[:,:crop_size, :crop_size,:]
pred  = load_tiff(os.path.join(inputdir, 'pred_disentangle_2404_D21-M3-S0-L8_6_1.tif'))
pred = pred[...,:crop_size, :crop_size]
pred.shape, gt.shape

In [ ]:
# gt = load_tiff('/facility/imganfacusers/Ashesh/NatureMethodsSegmentationOutputs/Analysis_2404_D21-m3-S0-L8_6/data_DDN/gt_ch2/GT-2.tif')
# gt_seg = load_tiff('/facility/imganfacusers/Ashesh/NatureMethodsSegmentationOutputs/Analysis_2404_D21-m3-S0-L8_6/data_DDN/gt_ch2/seg.tif')
# pred = load_tiff('/facility/imganfacusers/Ashesh/NatureMethodsSegmentationOutputs/Analysis_2404_D21-m3-S0-L8_6/data_DDN/pred_ch2/pred_ch2.tif')
# pred_seg = load_tiff('/facility/imganfacusers/Ashesh/NatureMethodsSegmentationOutputs/Analysis_2404_D21-m3-S0-L8_6/data_DDN/pred_ch2/seg.tif')


In [ ]:
input = gt[...,2]
gt_ch = gt[...,ch_idx]
pred_ch = pred[:,ch_idx]
raw_dict = {'input':input, 'pred':pred_ch, 'GT':gt_ch}


In [ ]:
from collections import defaultdict
def load_segmentation():
    # 2 level nested dictionary
    segmentation = defaultdict(lambda: defaultdict(list))
    for analystdir in os.listdir(segmentationdir):
        if os.path.isdir(os.path.join(segmentationdir,analystdir)):
            for subdir in [f'gt_ch{ch_idx+1}','superimposed',f'pred_ch{ch_idx+1}']:
                fpath = os.path.join(segmentationdir, analystdir, subdir, 'Seg.tif')
                assert os.path.exists(fpath), f'File {fpath} does not exist'
                if subdir.startswith('gt_ch'):
                    key = 'GT'
                elif subdir.startswith('pred_ch'):
                    key = 'pred'
                elif subdir.startswith('superimposed'):
                    key = 'input'
                else:
                    raise ValueError(f'Unknown subdir {subdir}')
                segmentation[analystdir][key].append(load_tiff(fpath)[..., :crop_size, :crop_size])
                if key == 'input':
                    if analystdir != 'data_JD':
                        segmentation[analystdir][key][-1] = segmentation[analystdir][key][-1] == ch_idx + 1
                    else:
                        segmentation[analystdir][key][-1] = segmentation[analystdir][key][-1] == (2*(ch_idx + 1) %3)
                print(analystdir, key, segmentation[analystdir][key][-1].shape)
    return segmentation

In [ ]:
seg_data = load_segmentation()
seg_data['data_JD']['input'][0] = seg_data['data_JD']['input'][0][None]
analysts  = list(seg_data.keys())
analysts

In [ ]:
for i in range(5):
    print(dice_coefficient(seg_data['data_JD']['GT'][0][i], seg_data['data_JMB']['GT'][0][i]))

In [ ]:
import matplotlib.pyplot as plt
_,ax = plt.subplots(figsize=(10,5),ncols=2)
ax[0].imshow(seg_data['data_JD']['GT'][0][i])
ax[1].imshow(seg_data['data_JMB']['GT'][0][i]
             )

In [ ]:
import matplotlib.pyplot as plt
from disentangle.analysis.plot_utils import clean_ax

ex_idx =0
transpose = False
save_to_file = False



ncols=4
nrows = 3
subres = 4
# how much do we want to reduce the width of the image
width_factor = 5/12
new_w = int(gt.shape[-2]*width_factor)
new_w += (gt.shape[-2] - new_w)%2
extra_w = (gt.shape[-2] - new_w)//2
imgsz = 3
_, ax = plt.subplots(figsize=(ncols*imgsz*width_factor, nrows*imgsz), ncols=ncols, nrows=nrows)

for d_idx, datatype in enumerate(['input','pred','GT']):
    raw_img = raw_dict[datatype][ex_idx]
    if transpose:
        raw_img = raw_img.T
    ax[d_idx,0].imshow(raw_img[:,extra_w:-extra_w][::subres,::subres], cmap='magma')
    for a_idx, analyst in enumerate(analysts):
        seg_img = seg_data[analyst][datatype][0][ex_idx]
        print(seg_img.shape)
        if transpose:
            seg_img = seg_img.T
        ax[d_idx, 1+a_idx].imshow(seg_img[:,extra_w:-extra_w][::subres,::subres], cmap='gray')
        # ax[d_idx, 1+a_idx].set_title(f'{analyst} {datatype}')

clean_ax(ax)
# remove subspaces between subplots
plt.subplots_adjust(wspace=0.02, hspace=0.02)
if save_to_file:
    model_token = os.path.basename(inputdir.strip('/'))
    fname = f'segmentation_3analyst_Idx{ex_idx}_{model_token}.png'
    fpath = os.path.join(OUTPUT_DIR, fname)
    print(fpath)
    plt.savefig(fpath, dpi = 100, bbox_inches='tight')


## Single reviewer prediction

In [ ]:
import numpy as np
def dice_coefficient(x,y):
    assert set(np.unique(x)) == set([0,1])
    assert set(np.unique(y)) == set([0,1])
    intersection = np.sum(x[y==1])
    union = np.sum(x) + np.sum(y)
    return 2*intersection/union

def add_text(ax, text, img_shape, place='TOP_LEFT'):
    """
    Adding text on image
    """
    assert place in ['TOP_LEFT', 'BOTTOM_RIGHT']
    if place == 'TOP_LEFT':
        ax.text(20, 40, text, bbox=dict(facecolor='white', alpha=0.9))
    elif place == 'BOTTOM_RIGHT':
        s0 = img_shape[1]
        s1 = img_shape[0]
        ax.text(s0 - s0 * 150 / 500, s1 - s1 * 35 / 500, text, bbox=dict(facecolor='white', alpha=0.9))


In [ ]:
from disentangle.analysis.plot_utils import clean_ax
import matplotlib.pyplot as plt

OneAnalyst_OUTPUT_DIR = OUTPUT_DIR.replace('three_analysts', 'one_analyst')
img_idx = 0
reviewer_key = 'data_JD'
seg_input = seg_data[reviewer_key]['input'][0][img_idx]
seg_pred = seg_data[reviewer_key]['pred'][0][img_idx]
seg_GT = seg_data[reviewer_key]['GT'][0][img_idx]

img_size = 5
hs = 200
he = 800

ws = 200
we = 800
save_to_file = False

_,ax = plt.subplots(figsize=(3*img_size, 2*img_size), ncols=3,nrows=2)
ax[0,0].imshow(raw_dict['input'][img_idx][hs:he,ws:we], cmap='gray')
ax[0,1].imshow(raw_dict['pred'][img_idx][hs:he,ws:we], cmap='gray')
ax[0,2].imshow(raw_dict['GT'][img_idx][hs:he,ws:we], cmap='gray')
ax[1,0].imshow(seg_input[hs:he,ws:we], cmap='gray')
ax[1,1].imshow(seg_pred[hs:he,ws:we], cmap='gray')
ax[1,2].imshow(seg_GT[hs:he,ws:we], cmap='gray')

dice_input = dice_coefficient(seg_GT.flatten() > 0, seg_input.flatten() >0)
dice_pred = dice_coefficient(seg_GT.flatten() > 0, seg_pred.flatten() >0)
add_text(ax[1,0], f'DICE: {dice_input:.2f}', seg_input.shape, place='TOP_LEFT')
add_text(ax[1,1], f'DICE: {dice_pred:.2f}', seg_input.shape, place='TOP_LEFT')


clean_ax(ax)
# remove the space between the subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)
if save_to_file:
    model_token = os.path.basename(inputdir.strip('/'))
    fname = f'segmentation_1analyst_reviewer:{reviewer_key}_Imgidx:{img_idx}_{model_token}_{hs}-{he}-{ws}-{we}.png'
    fpath = os.path.join(OneAnalyst_OUTPUT_DIR, fname)
    print(fpath)
    plt.savefig(fpath, dpi = 100, bbox_inches='tight')


In [ ]:
seg_data.keys()

In [ ]:
dice_score_dict= {}
reviewers = ['data_DDN','data_JMB', 'data_JD']
for review in reviewers:
    reviewer_seg = seg_data[review]
    gt_seg = reviewer_seg['GT'][0]
    pred_seg = reviewer_seg['pred'][0]
    input_seg = reviewer_seg['input'][0]
    dice_scores_pred = [dice_coefficient(gt.flatten() > 0, pred.flatten() > 0) for gt, pred in zip(gt_seg, pred_seg)]
    dice_scores_input = [dice_coefficient(gt.flatten() > 0, input.flatten() > 0) for gt, input in zip(gt_seg, input_seg)]

    dice_score_dict[review] = {'pred':dice_scores_pred, 
                               'input':dice_scores_input}

In [ ]:
import pandas as pd
pred_df = pd.DataFrame.from_dict({r:dice_score_dict[r]['pred'] for r in reviewers})
inp_df = pd.DataFrame.from_dict({r:dice_score_dict[r]['input'] for r in reviewers})

avg_pred = pred_df.mean(axis=0)
avg_inp = inp_df.mean(axis=0)

stderr_pred = pred_df.std(axis=0)/np.sqrt(pred_df.shape[0])
stderr_inp = inp_df.std(axis=0)/np.sqrt(inp_df.shape[0])

In [ ]:
import matplotlib.pyplot as plt
_,ax = plt.subplots(figsize=(8,4))
labels = avg_pred.index
x = np.arange(len(labels))/2
width = 0.1

rects1 = ax.bar(x - width/2, avg_pred, width, label='MicroSplit Prediction', yerr=stderr_pred, capsize=3, color='cyan', ecolor='black')
rects2 = ax.bar(x + width/2, avg_inp, width, label='MicrosSplit Input', yerr=stderr_inp, capsize=3, color='grey', ecolor='black')

ax.set_ylabel('Dice Similarity Coefficient')
ax.set_title('Segmentation Performance by Independent Analysts')
ax.set_xticks(x)
ax.set_xticklabels(['Analyst 1', 'Analyst 2'])
ax.legend(loc='upper right')
ax.set_ylim([0,1.1])


In [ ]:
pred_df

In [ ]:
import pandas as pd

dice_GT_score_dict= {}
for i1,an1 in enumerate(analysts):
    for i2,an2 in enumerate(analysts[i1+1:]):
        if an1 == an2:
            continue
        
        assert len(seg_data[an1]['GT']) == 1
        gt_seg1 = seg_data[an1]['GT'][0]
        gt_seg2 = seg_data[an2]['GT'][0]
        print(an1, an2)
        dice_GT_score_dict[f'A{i1}-A{i1+i2+1}'] = [dice_coefficient(gt1.flatten() > 0, gt2.flatten() > 0) for gt1, gt2 in zip(gt_seg1, gt_seg2)]
inter_reviewer_variability_df = pd.DataFrame.from_dict(dice_GT_score_dict)

In [ ]:
pred_df.columns = [f'A{i}' for i in range(3)]
pred_df

In [ ]:
import seaborn as sns

sns.boxplot(pd.concat([pred_df, inter_reviewer_variability_df], axis=1), fill=True)
# plt.ylim(0.7,0.9)
